In [27]:
# !pip install -qU transformers==4.39.3 accelerate bitsandbytes

In [28]:
from huggingface_hub import login

# Replace with your Hugging Face token
login(token="Token")

In [29]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
llm = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    use_auth_token=True
)
llm.eval()

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1010: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb):

In [30]:
# Define your input prompt
prompt = "Transulate it to Arabic 'Hello there!'"

# Tokenize the input prompt
inputs = tokenizer(prompt, return_tensors="pt").to(llm.device)

# Generate text using the model
outputs = llm.generate(**inputs,use_cache=False, )

# Decode and print the generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
generated_text

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


"Transulate it to Arabic 'Hello there!' \nTranslating 'Transulate' to Arabic means 'to translate'. \nTranslating '"

#Load a BOOK

In [31]:
!pip install pdfplumber

In [32]:
import pdfplumber

book_text = ""
with pdfplumber.open("/content/Ng_MachineLearningYearning.pdf") as pdf:
    for page in pdf.pages:
        t = page.extract_text() or ""
        book_text += t + "\n"


#Chunck it , Don't forget Overlapping

In [33]:
# Parameters
chunk_size = 800
overlap = 50

# Function to split text into overlapping chunks
def chunk_text(text, chunk_size=800, overlap=50):
    chunks = []
    start = 0
    text_length = len(text)

    while start < text_length:
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size - overlap  # move start but keep overlap

    return chunks

# Apply to your book text
book_chunks = chunk_text(book_text, chunk_size=800, overlap=50)

# Check the first few chunks
for i, c in enumerate(book_chunks[:3]):
    print(f"Chunk {i+1} (length {len(c)}):\n{c[:200]}...\n")


Chunk 1 (length 800):

Machine Learning Yearning is a
deeplearning.ai project.
© 2018 Andrew Ng. All Rights Reserved.
Page 2 Machine Learning Yearning-Draft Andrew Ng
Table of Contents
1 Why Machine Learning Strategy
2 How...

Chunk 2 (length 800):
te
14 Error analysis: Look at dev set examples to evaluate ideas
15 Evaluating multiple ideas in parallel during error analysis
16 Cleaning up mislabeled dev and test set examples
17 If you have a lar...

Chunk 3 (length 800):
g curves
29 Plotting training error
30 Interpreting learning curves: High bias
31 Interpreting learning curves: Other cases
32 Plotting learning curves
33 Why we compare to human-level performance
34 ...



#Embeddings

In [34]:
!pip install sentence-transformers

In [35]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained embedding model (you can choose others like 'all-MiniLM-L6-v2')
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for all chunks
book_embeddings = model.encode(book_chunks, show_progress_bar=True)

# Check first embedding
print(f"First chunk embedding length: {len(book_embeddings[0])}")

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

First chunk embedding length: 384


In [36]:
!pip install telebot

In [37]:
!pip install telebot -q
!pip install faiss-cpu -q

In [43]:
import telebot
import torch
import faiss
import numpy as np

# ----- 1️⃣ Telegram setup -----
bot_token = "8433007572:AAFqwSPW8TNNwLFTgFLZy4nKbnJdSGBWA7E"
bot = telebot.TeleBot(bot_token)

# ----- 2️⃣ Setup FAISS index (if not done already) -----
embedding_dim = book_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(np.array(book_embeddings, dtype=np.float32))

# ----- 3️⃣ RAG + LLM function -----
def rag_with_llm(query, top_k=3, max_new_tokens=200):
    # Embed query
    query_emb = model.encode([query])

    # Retrieve top-K chunks
    D, I = index.search(np.array(query_emb, dtype=np.float32), top_k)
    context_chunks = [book_chunks[i] for i in I[0]]

    # Build prompt
    context = "\n\n".join(context_chunks)

    prompt = f""" Using only the context below, answer the question concisely.
    Do not copy sentences verbatim.
    Do not add any information not in the context.
    If the answer is not in the context, write "Not available."
    Return a short, clear answer in your own words.

    Context:
    {context}

    Question:{query}
    Answer:"""

    # Tokenize and move to LLM device
    inputs = tokenizer(prompt, return_tensors="pt").to(llm.device)

    # Generate response
    with torch.no_grad():
        outputs = llm.generate(**inputs, max_new_tokens=max_new_tokens)

    # Decode and clean up
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Remove the prompt from the output
    if decoded.startswith(prompt):
        decoded = decoded[len(prompt):]

    # Clean each line but keep paragraphs
    lines = decoded.split("\n")
    clean_lines = [line.strip() for line in lines if line.strip() != ""]
    answer = "\n\n".join(clean_lines)
    return answer

# ----- 4️⃣ Telegram message handler -----
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    chat_id = message.chat.id
    user_query = message.text

    # Run RAG + LLM
    try:
        response = rag_with_llm(user_query, top_k=3, max_new_tokens=200)
    except Exception as e:
        response = f"Error: {str(e)}"

    # Send response back to Telegram
    bot.send_message(chat_id, response)

# ----- 5️⃣ Start bot -----
bot.infinity_polling()


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
2025-08-30 12:52:23,439 (__init__.py:1121 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
ERROR:TeleBot:Infinity polling: polling exited
2025-08-30 12:52:23,444 (__init__.py:1123 MainThread) ERROR - TeleBot: "Break infinity polling"
ERROR:TeleBot:Break infinity polling
